In [1]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [119]:
from google.colab import files
uploaded = files.upload()

Saving sansotei_google_review_2.xlsx to sansotei_google_review_2.xlsx


In [126]:
import pandas as pd

processed_data = []
df = pd.read_excel('sansotei_google_review_2.xlsx') # can also index sheet by name or fetch all sheets
mylist_review = df['Review'].tolist()
mylist_rating =df['Rating'].tolist()
for i in range(len(mylist_review)):
  y = [mylist_rating[i], mylist_review[i]]
  print(y)
  processed_data.append(y)    

[5, "This cozy restaurant seats about 35 guests, is brightly-lit, and has a moderate noise level. Sansotei hasn't had its grand opening yet, but the servers are very well-trained, and the operations seem to run smoothly. It was organized, with a waitlist for tables, and accurate estimates of the wait time. We received our food soon after ordering, and it was delicious!"]
[5, "For Mississauga, this is the best you'll get and closest to the real deal. The staff was extremely generous and welcoming. I had ordered the Tonkotsu Black which was extremely rich in flavour but not overly shadowed by salt (which most ramen shops mistakenly do most of the time).   The pork is extremely tender and the yolk of the eggs are always exactly what you'd expect....runny and delicious."]
[5, "Great Tonkotsu, a good rival for Kenzo for top Mississauga shop (not that we have alot of ramen places outside of toronto yet.) can't wait to try their Gyoza and Gyudon to"]
[3, 'The soup was a bit saltier than we ha

In [127]:
def split_content(content):
    # separate punctuations
    content = content.lower().replace(".", "") \
                 .replace(",", "") \
                 .replace(";", "") \
                 .replace("?", "") \
                 .replace("the", "") \
                 .replace("and", "")
    return content.split()

In [5]:
import torchtext
glove = torchtext.vocab.GloVe(name="6B", dim=50)

.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 399999/400000 [00:15<00:00, 26623.99it/s]


In [148]:
for i, line in enumerate(processed_data):
    if i > 30:
        break
    print("i", i)
    print(line)
    print(sum(int(w in glove.stoi) for w in split_content(line[-1])))
    print([glove.stoi[w] for w in split_content(line[-1]) if w in glove.stoi])

i 0
[5, "This cozy restaurant seats about 35 guests, is brightly-lit, and has a moderate noise level. Sansotei hasn't had its grand opening yet, but the servers are very well-trained, and the operations seem to run smoothly. It was organized, with a waitlist for tables, and accurate estimates of the wait time. We received our food soon after ordering, and it was delicious!"]
49
[37, 18707, 2551, 1578, 59, 1678, 4028, 14, 31, 7, 3230, 6056, 505, 40, 47, 33473, 875, 553, 34, 11731, 32, 191, 91190, 825, 1914, 4, 307, 10314, 20, 15, 2112, 17, 7, 10, 7221, 5697, 2886, 3, 2472, 79, 53, 496, 162, 565, 720, 49, 7487, 20, 15]
i 1
[5, "For Mississauga, this is the best you'll get and closest to the real deal. The staff was extremely generous and welcoming. I had ordered the Tonkotsu Black which was extremely rich in flavour but not overly shadowed by salt (which most ramen shops mistakenly do most of the time).   The pork is extremely tender and the yolk of the eggs are always exactly what you'd

In [130]:
def get_content_words(glove_vector):
    train, valid, test = [], [], []
    for i, line in enumerate(processed_data):
            content = line[-1]
            idxs = [glove_vector.stoi[w]        # lookup the index of word
                    for w in split_content(content)
                    if w in glove_vector.stoi] # keep words that has an embedding
            if not idxs: # ignore tweets without any word with an embedding
                continue
            idxs = torch.tensor(idxs) # convert list to pytorch tensor
            label = torch.tensor(int(line[0]) >= 3).long()
            if i % 5 < 3:
                train.append((idxs, label))
            elif i % 5 == 4:
                valid.append((idxs, label))
            else:
                test.append((idxs, label))
    return train, valid, test

train, valid, test = get_content_words(glove)

In [131]:
print(len(train))
print(len(valid))
print(len(test))

264
88
88


In [134]:
content, label = train[0]
print(content.shape)
print(label)

torch.Size([49])
tensor(1)


In [135]:
for i in range(10):
    tweet, label = train[i]
    print(tweet.shape)

torch.Size([49])
torch.Size([50])
torch.Size([26])
torch.Size([38])
torch.Size([66])
torch.Size([11])
torch.Size([154])
torch.Size([37])
torch.Size([28])
torch.Size([12])


In [170]:
glove_emb = nn.Embedding.from_pretrained(glove.vectors)

tweet_emb = glove_emb(tweet)
tweet_emb.shape

torch.Size([12, 50])

In [171]:
rnn_layer = nn.RNN(input_size=50,    # dimension of the input repr
                   hidden_size=50,   # dimension of the hidden units
                   batch_first=True) # input format is [batch_size, seq_len, repr_dim]

In [138]:
tweet_input = tweet_emb.unsqueeze(0)
out, last_hidden = rnn_layer(tweet_input)

In [139]:
print(tweet_input.shape)
print(out.shape)
print(last_hidden.shape)

torch.Size([1, 12, 50])
torch.Size([1, 12, 50])
torch.Size([1, 1, 50])


In [167]:
out2, last_hidden2 = rnn_layer(tweet_input)

In [168]:
print(out.shape)
print(last_hidden.shape)

torch.Size([1, 12, 50])
torch.Size([1, 1, 50])


In [169]:
print(last_hidden)
print(out[:,-1,:])

tensor([[[-0.4072, -0.3251, -0.0866,  0.0360,  0.3562,  0.3606, -0.3798,
           0.2625, -0.5767,  0.1671, -0.1327,  0.1684,  0.6406, -0.2407,
           0.3059,  0.3373,  0.5729,  0.2843, -0.1641, -0.1531, -0.1673,
          -0.3506,  0.2894,  0.0833,  0.5422,  0.1286, -0.5276,  0.1428,
           0.2566, -0.3278, -0.1027, -0.3933,  0.6071, -0.3087, -0.0087,
          -0.4118, -0.2665, -0.5288, -0.0642, -0.1792, -0.2775,  0.5165,
          -0.3258,  0.3501, -0.5520,  0.1992, -0.2143,  0.8334,  0.3290,
          -0.5494]]], grad_fn=<StackBackward0>)
tensor([[-0.4072, -0.3251, -0.0866,  0.0360,  0.3562,  0.3606, -0.3798,  0.2625,
         -0.5767,  0.1671, -0.1327,  0.1684,  0.6406, -0.2407,  0.3059,  0.3373,
          0.5729,  0.2843, -0.1641, -0.1531, -0.1673, -0.3506,  0.2894,  0.0833,
          0.5422,  0.1286, -0.5276,  0.1428,  0.2566, -0.3278, -0.1027, -0.3933,
          0.6071, -0.3087, -0.0087, -0.4118, -0.2665, -0.5288, -0.0642, -0.1792,
         -0.2775,  0.5165, -0.3258, 

In [164]:
class contentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(contentRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Forward propagate the RNN
        out, _ = self.rnn(x)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

model = contentRNN(50, 50, 2)

In [165]:
content, label = test[1]
content = content.unsqueeze(0)
print(processed_data[8])
print(content)
print(label)
print(model(content).max(1, keepdim=True)[1])

[2, 'Good service, but the broth was quite salty, and the meat had a weird taste, I’m not sure if that’s the style of the restaurant, but I’m not really a fan.']
tensor([[  219,   270,    34, 19871,    15,  1689, 23670,  3806,    40,     7,
         10267,  4374,    36,  1085,    83,  1135,     3,  2551,    34,    36,
           588,     7,  3267]])
tensor(0)
tensor([[0]])


In [166]:
content, label = test[0]
content = content.unsqueeze(0)
print(processed_data[3])
print(content)
print(label)
print(model(content).max(1, keepdim=True)[1])

[3, 'The soup was a bit saltier than we had expected. Will probably come back after things have settled down and they have their official grand opening.']
tensor([[  8233,     15,      7,   1594, 122217,     73,     53,     40,    287,
             43,    965,    326,    137,     49,    654,     33,   2845,    135,
           3524,     33,  29764,    239,  33473,    875]])
tensor(1)
tensor([[1]])
